# Vertex AI Search with Filters & Metadata

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/search/search_filters_metadata.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/search_filters_metadata.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/search/search_filters_metadata.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>


---


* Authors: Ruchika Kharwar
* Created: 5 Sep 2023

---

## Objective

This notebook shows how to use [filters and metadata](https://cloud.google.com/generative-ai-app-builder/docs/filter-search-metadata) in search requests to [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/introduction).

This works with unstructrued apps that contain metadata. You can use metadata fields to restrict your search to a specific set of documents.


Services used in the notebook:

- ✅ Vertex AI Search for document search and retrieval

## Install pre-requisites

If running in Colab install the pre-requisites into the runtime. Otherwise it is assumed that the notebook is running in Vertex AI Workbench. In that case it is recommended to install the pre-requistes from a terminal using the `--user` option.


In [2]:
%pip install -q google-cloud-discoveryengine==0.11.2 --upgrade --user



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>



## Authenticate

If running in Colab authenticate with `google.colab.google.auth` otherwise assume that running on Vertex AI Workbench.


In [4]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

from google.auth import default
creds, _ = default()


## Configure notebook environment


## Data store metadata

Metadata for the data store `<DATA_STORE_ID>` looks like this:

```json
{
  "id": "1",
  "structData": {
    "title": "Document1",
    "category": [
      "PersonaA"
    ],
    "name": "Document1"
  },
  "content": {
    "mimeType": "application/pdf",
    "uri": "gs://<BUCKETNAME>/data/Document1"
  }
}
```

```json
{
  "id": "2",
  "structData": {
    "title": "Document2",
    "category": [
      "PersonaA",
      "PersonaB"
    ],
    "name": "Document2"
  },
  "content": {
    "mimeType": "application/pdf",
    "uri": "gs://<BUCKETNAME>/data/Document2"
  }
}
```

### Set the following constants to reflect your environment

In [ ]:
PROJECT_ID = "<PROJECT_ID>"
LOCATION = "global" # Replace with your data store location
DATA_STORE_ID = "<DATA_STORE_ID>"


### REST API examples

The filter `name: ANY("Document1")` ensures the query is against only the documents with `name` matching `Document1`.

In [ ]:
%%bash -s "$PROJECT_ID" "$LOCATION" "$DATA_STORE_ID"

project_id=$1
location=$2
data_store_id=$3

curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://discoveryengine.googleapis.com/v1beta/projects/$project_id/locations/$location/collections/default_collection/dataStores/$data_store_id/servingConfigs/default_search:search" \
-d '{
"query": "claim",
"filter": "name: ANY(\"Document1\")"
}'


The filter `category: ANY("PersonaB")` ensures the query is against only the documents with `name` matching `Document1`.

In [ ]:
%%bash -s "$PROJECT_ID" "$LOCATION" "$DATA_STORE_ID"

project_id=$1
location=$2
data_store_id=$3

curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://discoveryengine.googleapis.com/v1beta/projects/$project_id/locations/$location/collections/default_collection/dataStores/$data_store_id/servingConfigs/default_search:search" \
-d '{
"query": "claims",
"filter": "category: ANY(\"PersonaB\")"
}'


### Python code equivalent

In [ ]:
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1beta as discoveryengine

def search_data_store(
    project_id: str,
    location: str,
    data_store_id: str,
    search_query: str,
    filter_str: str
) -> discoveryengine.SearchResponse:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search engine serving config
    # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
    serving_config = client.serving_config_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        serving_config="default_config",
    )

    # Optional: Configuration options for search
    # Refer to the `ContentSearchSpec` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
            max_extractive_answer_count=5,
            max_extractive_segment_count=1,
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=False,
            ignore_non_summary_seeking_query=False,
        ),
    )

    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        filter=filter_str,
        page_size=5,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    return response


In [ ]:
search_query = "how to process a claim"
filter_str = "name: ANY(\"Document1\")"

results = search_data_store(PROJECT_ID, LOCATION, DATA_STORE_ID, search_query, filter_str)

print(f"\nQuestion: '{search_query}'\n\n")
print("Summary" + "-"*40)
print(results.summary.summary_text)

print("Raw Results" + "-"*40)
print(results)


Here is a slightly more complex filter based on 2 metadata values

In [ ]:
search_query = "how to process a claim"
filter_str = "name: ANY(\"Document1\") AND category: ANY(\"PersonaA\")"

results = search_data_store(PROJECT_ID, LOCATION, DATA_STORE_ID, search_query, filter_str)

print(f"\nQuestion: '{search_query}'\n\n")
print("Summary" + "-"*40)
print(results.summary.summary_text)

print("Raw Results" + "-"*40)
print(results)
